In [1]:
import websocket
# from websocket import create_connection

import simplejson as json


In [2]:

ws = websocket.create_connection("wss://api.tiingo.com/fx")

subscribe = {
        'eventName':'subscribe',
        'authorization':'ebed6535a2acbe1a06b0057e8d1e8af61e5cefa7',
        'eventData': {
            'thresholdLevel': 5,
            'tickers': ['eurusd']
    }
}

ws.send(json.dumps(subscribe))
while True:
    print(ws.recv())

{"response": {"code": 200, "message": "Success"}, "data": {"subscriptionId": 17270378}, "messageType": "I"}
{"response": {"code": 200, "message": "HeartBeat"}, "messageType": "H"}
{"service":"fx","messageType":"A","data":["Q","eurusd","2024-05-08T22:24:18.649000+00:00",1000000.0,1.07494,1.07499,1000000.0,1.07504]}
{"service":"fx","messageType":"A","data":["Q","eurusd","2024-05-08T22:24:19.726000+00:00",1000000.0,1.07494,1.074985,1000000.0,1.07503]}
{"service":"fx","messageType":"A","data":["Q","eurusd","2024-05-08T22:24:20.937000+00:00",1000000.0,1.07494,1.074985,1000000.0,1.07503]}
{"service":"fx","messageType":"A","data":["Q","eurusd","2024-05-08T22:24:22.113000+00:00",1000000.0,1.07494,1.074985,1000000.0,1.07503]}
{"service":"fx","messageType":"A","data":["Q","eurusd","2024-05-08T22:24:23.348000+00:00",1000000.0,1.07494,1.074985,1000000.0,1.07503]}
{"service":"fx","messageType":"A","data":["Q","eurusd","2024-05-08T22:24:24.637000+00:00",1000000.0,1.07494,1.074985,1000000.0,1.07503]}

KeyboardInterrupt: 